In [1]:
def ssp(subject, raw, report_path, save_path):
    import mne
    from mne.report import Report
    import matplotlib.pyplot as plt
    import matplotlib
    matplotlib.use('Agg')
    from mne.preprocessing import create_eog_epochs, create_ecg_epochs, \
                                 compute_proj_ecg, compute_proj_eog

    n_jobs = 16
    reject = {'mag': 6e-12, 'grad': 20e-11}
    flat = {'mag': 4e-14, 'grad': 7e-13}
    report = Report(image_format='png')

    raw.load_data()
    raw.filter(None, 110., fir_design='firwin', n_jobs=n_jobs)
    raw.notch_filter([50, 100], filter_length='auto', phase='zero', n_jobs=n_jobs)

    # Detecting EOG/ECG channels (robust: Do not raise if missing)
    eog_picks = mne.pick_types(raw.info, eog=True)
    if len(eog_picks) == 0:
        vEOG = None
        hEOG = None
    elif len(eog_picks) == 1:
        vEOG = raw.info['ch_names'][eog_picks[0]]
        hEOG = None
    else:
        vEOG = raw.info['ch_names'][eog_picks[0]]
        hEOG = raw.info['ch_names'][eog_picks[1]]

    ecg_picks = mne.pick_types(raw.info, ecg=True)
    if len(ecg_picks) == 0:
        ecg_present = False
    else:
        ecg_present = True

    # Initializing the projector containers
    projs_veog = []
    projs_heog = []
    projs_ecg = []

    # Computing the EOG projectors (if channels are present)
    if vEOG is not None:
        projs_veog, events_veog = compute_proj_eog(
            raw, ch_name=vEOG,
            n_grad=1, n_mag=1, n_eeg=0,
            reject=reject, flat=flat, n_jobs=n_jobs
        )
        if len(events_veog) > 0:
            try:
                eog_evoked = create_eog_epochs(raw, ch_name=vEOG).average()
                if eog_evoked.nave > 0:
                    eog_evoked.del_proj().add_proj(projs_veog)
                    fig, axes = plt.subplots(1, 2, figsize=(8, 3), squeeze=False)
                    for pi, proj in enumerate((False, True)):
                        eog_evoked.plot(picks='mag', proj=proj, axes=axes[:, pi], spatial_colors=True)
                        if pi == 0:
                            for ax in axes[:, pi]:
                                parts = ax.get_title().split('(')
                                ax.set(ylabel=f'{parts[0]} ({ax.get_ylabel()})\n'
                                              f'{parts[1].replace(")", "")}')
                        axes[0, pi].set(title=f'proj={proj}')
                        for text in list(axes[0, pi].texts):
                            text.remove()
                    plt.setp(axes[1:, :].ravel(), title='')
                    plt.setp(axes[:, 1:].ravel(), ylabel='')
                    plt.setp(axes[:-1, :].ravel(), xlabel='')
                    mne.viz.tight_layout()
                    report.add_figure(fig, caption='SSP proj vEOG', title='SSP')
            except Exception:
                # plotting/epoching may fail for some datasets; continue gracefully
                pass

    if hEOG is not None:
        projs_heog, events_heog = compute_proj_eog(
            raw, ch_name=hEOG,
            n_grad=1, n_mag=1, n_eeg=0,
            reject=reject, flat=flat, n_jobs=n_jobs
        )
        if len(events_heog) > 0:
            try:
                eog_evoked = create_eog_epochs(raw, ch_name=hEOG).average()
                if eog_evoked.nave > 0:
                    eog_evoked.del_proj().add_proj(projs_heog)
                    fig, axes = plt.subplots(1, 2, figsize=(8, 3), squeeze=False)
                    for pi, proj in enumerate((False, True)):
                        eog_evoked.plot(picks='mag', proj=proj, axes=axes[:, pi], spatial_colors=True)
                        if pi == 0:
                            for ax in axes[:, pi]:
                                parts = ax.get_title().split('(')
                                ax.set(ylabel=f'{parts[0]} ({ax.get_ylabel()})\n'
                                              f'{parts[1].replace(")", "")}')
                        axes[0, pi].set(title=f'proj={proj}')
                        for text in list(axes[0, pi].texts):
                            text.remove()
                    plt.setp(axes[1:, :].ravel(), title='')
                    plt.setp(axes[:, 1:].ravel(), ylabel='')
                    plt.setp(axes[:-1, :].ravel(), xlabel='')
                    mne.viz.tight_layout()
                    report.add_figure(fig, caption='SSP proj hEOG', title='SSP')
            except Exception:
                pass

    # Computing ECG projectors (if ECG present)
    if ecg_present:
        projs_ecg, events_ecg = compute_proj_ecg(
            raw, n_grad=2, n_mag=2, n_eeg=0,
            reject=reject, flat=flat, n_jobs=n_jobs
        )
        if len(events_ecg) > 0:
            try:
                ecg_evoked = create_ecg_epochs(raw).average()
                if ecg_evoked.nave > 0:
                    ecg_evoked.del_proj().add_proj(projs_ecg)
                    fig, axes = plt.subplots(1, 2, figsize=(8, 3), squeeze=False)
                    for pi, proj in enumerate((False, True)):
                        ecg_evoked.plot(picks='mag', proj=proj, axes=axes[:, pi], spatial_colors=True)
                        if pi == 0:
                            for ax in axes[:, pi]:
                                parts = ax.get_title().split('(')
                                ax.set(ylabel=f'{parts[0]} ({ax.get_ylabel()})\n'
                                              f'{parts[1].replace(")", "")}')
                        axes[0, pi].set(title=f'proj={proj}')
                        for text in list(axes[0, pi].texts):
                            text.remove()
                    plt.setp(axes[1:, :].ravel(), title='')
                    plt.setp(axes[:, 1:].ravel(), ylabel='')
                    plt.setp(axes[:-1, :].ravel(), xlabel='')
                    mne.viz.tight_layout()
                    report.add_figure(fig, caption='SSP proj ECG', title='SSP')
            except Exception:
                pass

    # Add whatever projectors were computed and visualize
    all_projs = []
    if projs_ecg:
        all_projs.extend(projs_ecg)
    if projs_veog:
        all_projs.extend(projs_veog)
    if projs_heog:
        all_projs.extend(projs_heog)

    for title in ('Without', 'With'):
        if title == 'With' and all_projs:
            raw.add_proj(all_projs)
        with mne.viz.use_browser_backend('matplotlib'):
            fig = raw.plot(n_channels=30, start=30, duration=20.0, scalings=dict(grad=2e-11))
        fig.subplots_adjust(top=0.9)
        fig.suptitle(f'{title} EOG and ECG projectors', size='xx-large', weight='bold')
        report.add_figure(fig, caption=f'{title} EOG and ECG projectors', title='SSP')

    plt.close('all')

    # apply projectors if any
    if all_projs:
        raw.apply_proj()

    report.save(report_path, open_browser=False, overwrite=True)
    raw.save(save_path, overwrite=True)



In [21]:
import mne
import glob
import mne_bids

subjects_names = ['Z201']

bids_root = r'D:\\ds005234\\'
bids_derivatives = bids_root + r'derivatives\\'

task = 'vowels'

for subject in subjects_names:

    bids_path = mne_bids.BIDSPath(
        subject=subject,
        root=bids_derivatives,
        task=task,
        datatype='meg',
        processing='tsss+mc+transtooptimal'
    )

    files = glob.glob(str(bids_path.directory) + '/*.fif')
    files = [x for x in files if 'optimal' in x and task in x and 'ssp' not in x]
    files = sorted(files)

    if len(files) == 0:
        raise RuntimeError(f'No matching FIF files found for {subject}')

    raw_list = [mne.io.read_raw_fif(f) for f in files]
    raw = mne.io.concatenate_raws(raw_list, on_mismatch='ignore')

    bp_annot = mne_bids.BIDSPath(
        subject=subject,
        root=bids_derivatives,
        task=task,
        processing='tsss+mc+transtooptimal',
        extension='.pos'
    )

    annot = mne.read_annotations(str(bp_annot.fpath) + '_annotations.txt')
    raw.set_annotations(annot)

    bp = mne_bids.get_bids_path_from_fname(files[-1])

    save_path = bp.copy().update(
        run=None,
        processing='tsss+mc+transtooptimal+0110hz+ssp',
        root=bids_derivatives
    )
    save_path.mkdir()

    report_path = str(save_path.fpath)[:-8] + '-report.html'
    save_path = str(save_path.fpath)

    ssp(subject, raw, report_path, save_path)



Opening raw data file D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_run-01_proc-tsss+mc+transtooptimal_meg.fif...
    Range : 45000 ... 582999 =     45.000 ...   582.999 secs
Ready.
Opening raw data file D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_run-02_proc-tsss+mc+transtooptimal_meg.fif...
    Range : 24000 ... 564999 =     24.000 ...   564.999 secs
Ready.
Opening raw data file D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_run-03_proc-tsss+mc+transtooptimal_meg.fif...
    Range : 37000 ... 579999 =     37.000 ...   579.999 secs
Ready.
Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min

C:\Users\hp\AppData\Local\Temp\ipykernel_21624\2685967882.py:193: RuntimeWarning: Unknown processing history item 220
  raw_list = [mne.io.read_raw_fif(f) for f in files]
C:\Users\hp\AppData\Local\Temp\ipykernel_21624\2685967882.py:193: RuntimeWarning: Unknown processing history item 220
  raw_list = [mne.io.read_raw_fif(f) for f in files]
C:\Users\hp\AppData\Local\Temp\ipykernel_21624\2685967882.py:193: RuntimeWarning: Unknown processing history item 220
  raw_list = [mne.io.read_raw_fif(f) for f in files]
C:\Users\hp\AppData\Local\Temp\ipykernel_21624\2685967882.py:193: RuntimeWarning: Unknown processing history item 220
  raw_list = [mne.io.read_raw_fif(f) for f in files]
C:\Users\hp\AppData\Local\Temp\ipykernel_21624\2685967882.py:193: RuntimeWarning: Unknown processing history item 220
  raw_list = [mne.io.read_raw_fif(f) for f in files]
C:\Users\hp\AppData\Local\Temp\ipykernel_21624\2685967882.py:193: RuntimeWarning: Unknown processing history item 220
  raw_list = [mne.io.read_r

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 1.1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 110.00 Hz
- Upper transition bandwidth: 27.50 Hz (-6 dB cutoff frequency: 123.75 Hz)
- Filter length: 121 samples (0.121 s)



[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 tasks      | elapsed:    2.1s
[Parallel(n_jobs=16)]: Done  56 tasks      | elapsed:    4.2s
[Parallel(n_jobs=16)]: Done 146 tasks      | elapsed:    7.3s
[Parallel(n_jobs=16)]: Done 272 tasks      | elapsed:   11.6s
[Parallel(n_jobs=16)]: Done 306 out of 306 | elapsed:   12.6s finished


Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6601 samples (6.601 s)



...

Using matplotlib as 2D backend.


C:\Users\hp\anaconda3\Lib\site-packages\mne\viz\utils.py:158: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  (fig or plt).show(**kwargs)


4 projection items deactivated


C:\Users\hp\anaconda3\Lib\site-packages\mne\viz\utils.py:158: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  (fig or plt).show(**kwargs)


Created an SSP operator (subspace dimension = 4)
4 projection items activated
SSP projectors applied...
Saving report to : D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_proc-tsss+mc+transtooptimal+0110hz+ssp-report.html
Writing D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_proc-tsss+mc+transtooptimal+0110hz+ssp_meg.fif
Closing D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_proc-tsss+mc+transtooptimal+0110hz+ssp_meg.fif
Writing D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_proc-tsss+mc+transtooptimal+0110hz+ssp_meg-1.fif
Closing D:\ds005234\derivatives\sub-Z201\meg\sub-Z201_task-vowels_proc-tsss+mc+transtooptimal+0110hz+ssp_meg-1.fif
[done]
